# Windninja Pipeline

## 1. Import & install neccessary packages

In [1]:
import os
import subprocess
from pathlib import Path
from datetime import datetime, timedelta, timezone
from dateutil import tz
import pandas as pd
from sqlalchemy import create_engine, text
import shutil, subprocess


## 2. Set variables

In [2]:
WINDNINJA_CLI=Path(r"C:\Program Files (x86)\WindNinja\WindNinja-3.12.0\bin\WindNinja_cli.exe")
BASE_OUT = Path("windninja_output")

In [3]:
# ------------------ USER SETTINGS ------------------

# DEM used by WindNinja
ELEVATION_FILE = r"merlingen_7.5mi.tif"

# Vegetation & heights
VEGETATION = "trees"           # grass | trees | brush (set to your dominant cover)
OUTPUT_WIND_HEIGHT = 10.0     # meters AGL for output
OUTPUT_SPEED_UNITS = "mps"     # m/s | mph | kph
TIMEZONE = "Europe/Zurich"     # must be a valid IANA tz string


# Mesh resolution (meters)
MESH_RES_M = 150.0


In [4]:

# Date/time window to process 
START = datetime(2025, 10, 30, 18, 0, tzinfo=tz.gettz(TIMEZONE))
END   = datetime(2025, 10, 30, 18, 50, tzinfo=tz.gettz(TIMEZONE)) 

# 6 timesteps (local) within this hour:
NR_TIMESTEPS = 6 

In [5]:
# Connect to wind database

engine = create_engine(
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}"
    f"@{os.getenv('DB_HOST')}:{int(os.getenv('DB_PORT', 3306))}/{os.getenv('DB_NAME')}",
    pool_pre_ping=True,
    future=True,
)

# Three station views from database
VIEWS = ["v_windninja_export_fru", "v_windninja_export_int", "v_windninja_export_thu"] 

# time in views CET/CEST automatically handled
LOCAL_TZ = tz.gettz("Europe/Zurich")

# The exact column order you said the view provides:
WN_HEADERS = [
    "Station_Name","Coord_Sys(PROJCS,GEOGCS)","Datum(WGS84,NAD83,NAD27)",
    "Lat/YCoord","Lon/XCoord","Height","Height_Units(meters,feet)",
    "Speed","Speed_Units(mph,kph,mps,kts)","Direction(degrees)",
    "Temperature","Temperature_Units(F,C)","Cloud_Cover(%)",
    "Radius_of_Influence","Radius_of_Influence_Units(miles,feet,meters,km)",
    "date_time"
]

In [6]:
try:
    with engine.connect() as connection:
        print("✅ Connection successful!")
except Exception as e:
    print("❌ Connection failed!")
    print(e)

✅ Connection successful!


## 2. Define Helper Functions

Source: https://github.com/gagreene/WindNinja/  
Source: https://github.com/firelab/windninja

In [7]:
def _local_day_to_utc_bounds(any_local_dt: datetime):
    """
    Given a LOCAL datetime, return UTC bounds covering the UTC calendar day
    that has the *same Y-M-D* as the local date.

    Example:
      input  2025-08-31 06:00+02 -> returns
      start_utc = 2025-08-31 00:00:00Z (naive)
      end_utc   = 2025-09-01 00:00:00Z (naive, exclusive)
    """
    # Ensure we interpret the argument in local time (handle naive safely)
    if any_local_dt.tzinfo is None:
        local_dt = any_local_dt.replace(tzinfo=LOCAL_TZ)
    else:
        local_dt = any_local_dt.astimezone(LOCAL_TZ)

    # Take the *local* Y-M-D fields, but build a UTC-day window with the same Y-M-D
    y, m, d = local_dt.year, local_dt.month, local_dt.day
    start_utc_dt = datetime(y, m, d, 0, 0, tzinfo=timezone.utc)
    end_utc_dt   = start_utc_dt + timedelta(days=1)

    # Return naive datetimes for MySQL DATETIME comparison
    return start_utc_dt.replace(tzinfo=None), end_utc_dt.replace(tzinfo=None)

In [8]:
def _station_code(view: str) -> str:
    v = view.lower()
    if "fru" in v: return "FRU"
    if "int" in v: return "INT"
    if "thu" in v: return "THU"
    return view.split("_")[-1].upper()

In [9]:
def make_daily_station_filename(view: str, start_local: datetime, end_local: datetime = None, index: int = None) -> str:
    """
    Build: THU-CET-YYYY-MM-DD_0000-yyyy-mm-dd_0000-{index}.csv

    Back-compat:
      - If called with only (view, start_local) as your fetcher does,
        we auto-fill:
          * end_local := global END if present, else start_local
          * index     := stable index from VIEWS order (THU/INT/FRU etc.)
    """
    # infer end_local if not provided (use global END if available)
    if end_local is None:
        end_local = globals().get("END", start_local)

    # infer index if not provided: stable position in VIEWS
    if index is None:
        try:
            index = globals()["VIEWS"].index(view)
        except Exception:
            index = 0

    code      = _station_code(view)
    tz_label  = "CET"
    start_str = start_local.strftime("%Y-%m-%d_0000")
    end_str   = end_local.strftime("%Y-%m-%d_0000")
    return f"{code}-{tz_label}-{start_str}-{end_str}-{index}.csv"

In [10]:
def fetch_write_daily_station_csvs(engine, out_dir: Path, start_local: datetime) -> dict:
    """
    For each view:
      - Determine the LOCAL day of `start_local`
      - Build UTC bounds for that day
      - Query MySQL by UTC (date_time is UTC in the view)
      - DO NOT CONVERT `date_time` (leave exactly as stored)
      - Enforce column order (names unchanged)
      - Write one CSV per station/day using your naming pattern
    Returns {view: csv_path}.
    """
    out_dir.mkdir(parents=True, exist_ok=True)

    # the local 'day' is derived from the user-provided START (local)
    day_local = start_local.astimezone(LOCAL_TZ).replace(hour=0, minute=0, second=0, microsecond=0)
    start_utc, end_utc = _local_day_to_utc_bounds(day_local)

    sql = """
      SELECT *
      FROM `{view}`
      WHERE `date_time` >= :a AND `date_time` < :b
      ORDER BY `date_time` ASC
    """

    written = {}
    with engine.begin() as conn:
        for view in VIEWS:
            df = pd.read_sql(text(sql.format(view=view)), conn,
                             params={"a": start_utc, "b": end_utc})

            if df.empty:
                # no file written for this view/day
                continue

            # (Important) Do NOT touch df["date_time"]: leave it exactly as stored in DB (UTC)
            # Ensure headers exist and enforce order; this does not change values.
            missing = [c for c in WN_HEADERS if c not in df.columns]
            if missing:
                raise ValueError(f"{view}: missing expected columns: {missing}")
            df = df[WN_HEADERS]

            # Write the CSV with values exactly as read (no datetime conversion/formatting)
            fname = make_daily_station_filename(view, day_local)
            csv_path = out_dir / fname
            df.to_csv(csv_path, index=False)
            written[view] = csv_path

    return written

In [11]:
def ensure_daily_files(engine, base_out: Path, start_local: datetime, end_local: datetime, views) -> tuple[Path, dict]:
    """
    Create the WXSTATIONS folder and guarantee exactly one CSV per station/day.
    - Reuse existing files (no overwrite).
    - Only create missing ones by calling the unchanged fetcher into a temp folder,
      then moving across the missing files to the target folder.
    Returns (day_folder, {view: path_str})
    """
    tz_label   = "CET"
    folderName = f"WXSTATIONS-{tz_label}-{start_local:%Y-%m-%d_0000}-{end_local:%Y-%m-%d_0000}-lake-thun"
    day_folder = base_out / folderName
    day_folder.mkdir(parents=True, exist_ok=True)

    # Expected filenames (stable indices)
    expected = {}
    for i, view in enumerate(views):
        fname = make_daily_station_filename(view, start_local, end_local, i)
        expected[view] = day_folder / fname

    # What already exists?
    have = {v: p for v, p in expected.items() if p.exists()}
    missing_views = [v for v in views if not expected[v].exists()]

    if not missing_views:
        # Nothing to write; return existing mapping
        return day_folder, {v: str(expected[v]) for v in views if expected[v].exists()}

    # Write all station CSVs for the day into a TEMP folder (using the unchanged fetcher)
    tmp_dir = day_folder / "_tmp_station_write"
    tmp_dir.mkdir(parents=True, exist_ok=True)

    # The fetcher will:
    #  * compute day_local = midnight of START (local)
    #  * call make_daily_station_filename(view, day_local)  <-- now back-compat & stable
    tmp_written = fetch_write_daily_station_csvs(engine, tmp_dir, start_local)  # returns {view: Path}

    # Move only MISSING views into the final folder; leave existing files untouched
    for view in missing_views:
        src = Path(tmp_written.get(view, ""))
        if not src or not src.exists():
            # No data fetched for that view/day; skip
            continue
        dst = expected[view]
        # Ensure the src name matches the expected final name; if not, rename while moving
        if src.name != dst.name:
            shutil.move(str(src), str(dst))
        else:
            shutil.move(str(src), str(dst))

    # Cleanup temp dir if empty
    try:
        tmp_dir.rmdir()
    except OSError:
        pass  # not empty (e.g., some views had no data or something else wrote there)

    # Build final mapping of what now exists
    result = {v: str(p) for v, p in expected.items() if p.exists()}
    return day_folder, result

In [12]:
# Prepare a list of stations csvs

def create_station_list_csv(day_folder: Path, start_local: datetime, end_local: datetime) -> Path:
    """
    Writes a one-column CSV:
      Station_File_List,
      <filenames for THIS day window only>
    """
    start_str = start_local.strftime("%Y-%m-%d_0000")
    end_str   = end_local.strftime("%Y-%m-%d_0000")

    # Only include files matching this day's window and exclude the list file itself
    station_files = sorted([
        f.name for f in day_folder.glob("*.csv")
        if f.name != f"station_file_list_{start_local:%Y%m%d}.csv"
        and f"-CET-{start_str}-{end_str}-" in f.name
    ])

    if not station_files:
        raise FileNotFoundError(f"No station CSVs for {start_str} → {end_str} in {day_folder}")

    out_csv = day_folder / f"station_file_list_{start_local:%Y%m%d}.csv"
    with open(out_csv, "w", encoding="utf-8", newline="\n") as fh:
        fh.write("Station_File_List,\n")
        for name in station_files:
            fh.write(f"{name}\n")

    return out_csv

In [13]:
def _as_posix(p):
    # accepts Path or str, returns a forward-slash path (works on Windows)
    return str(Path(p)).replace("\\", "/")

In [14]:
def build_cfg_point_init_span_cli(cfg_path, output_dir, station_csv, start_local, stop_local, n_steps):
    """
    WindNinja CLI — pointInitialization, NO diurnal, MULTI-step run over [start_local, stop_local],
    with number_time_steps = n_steps (evenly spaced across the span).
    """
    lines = [
        f"num_threads = {os.cpu_count() or 2}",
        f"elevation_file = {_as_posix(ELEVATION_FILE)}",
        "initialization_method = pointInitialization",
        f"wx_station_filename = {_as_posix(station_csv)}",
        "match_points = true",

        # CLI inputs analogous to what the GUI collects:
        f"number_time_steps = {int(n_steps)}",

        f"start_year = {start_local.year}",
        f"start_month = {start_local.month}",
        f"start_day = {start_local.day}",
        f"start_hour = {start_local.hour}",
        f"start_minute = {start_local.minute}",

        f"stop_year = {stop_local.year}",
        f"stop_month = {stop_local.month}",
        f"stop_day = {stop_local.day}",
        f"stop_hour = {stop_local.hour}",
        f"stop_minute = {stop_local.minute}",

        f"time_zone = {TIMEZONE}",
        f"vegetation = {VEGETATION}",
        "diurnal_winds = false",

        f"mesh_resolution = {int(MESH_RES_M)}",
        "units_mesh_resolution = m",
        f"output_wind_height = {float(OUTPUT_WIND_HEIGHT)}",
        "units_output_wind_height = m",
        F"output_speed_units = {OUTPUT_SPEED_UNITS}",

        f"output_path = {_as_posix(output_dir)}",
        "write_goog_output = true",
        "write_ascii_output = true",
        "write_farsite_atm = false",
    ]
        
    cfg_path.write_text("\n".join(lines), encoding="utf-8")

In [15]:
# --- hour folder (unchanged, just uses your mesh var) ---

def make_hour_folder(base_out: Path, dt_local) -> Path:
    """Folder name: Output_YYYY-MM-DD_HH00-HH50_Google earth_Fire_150m_10min"""
    start_str = dt_local.strftime("%Y-%m-%d_%H00")
    end_str = dt_local.strftime("%H50")
    name = f"Output_{start_str}-{end_str}_Google earth_Fire_{int(MESH_RES_M)}m_10min"
    path = base_out / name
    path.mkdir(parents=True, exist_ok=True)
    return path

In [16]:
def make_span_folder(base_out: Path, start_local, stop_local) -> Path:
    name = f"Output_{start_local:%Y-%m-%d_%H%M}-{stop_local:%H%M}_Google earth_Fire_{int(MESH_RES_M)}m_10min"
    path = base_out / name
    path.mkdir(parents=True, exist_ok=True)
    return path

In [17]:
def run_windninja(cfg_path: Path) -> str:
    """
    Execute WindNinja CLI with a direct full path.
    """
    if not WINDNINJA_CLI.exists():
        raise FileNotFoundError(f"WindNinja CLI not found at: {WINDNINJA_CLI}")

    cmd = [str(WINDNINJA_CLI), str(cfg_path)]

    # Run process, capture stdout/stderr
    result = subprocess.run(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        shell=False,
    )

    if result.returncode != 0:
        raise RuntimeError(
            f"WindNinja failed ({result.returncode}). Output:\n{result.stdout}"
        )

    return result.stdout

## 3. Execute the program

In [19]:
def main():
    BASE_OUT.mkdir(parents=True, exist_ok=True)

    # Ensure exactly one CSV per station/day; do not overwrite existing
    day_folder, daily_csvs = ensure_daily_files(engine, BASE_OUT, START, END, VIEWS)

    # Station list for THIS day only
    station_list_csv = create_station_list_csv(day_folder, START, END)

    # Output folder for the simulation span
    out_folder = make_span_folder(BASE_OUT, START, END)
    out_folder.mkdir(parents=True, exist_ok=True)

    # Build cfg — pointInitialization using the list CSV
    cfg_path = out_folder / f"wn_ALL_{START:%Y%m%d_%H%M}-{END:%Y%m%d_%H%M}.cfg"
    build_cfg_point_init_span_cli(
        cfg_path,
        out_folder,
        station_list_csv,
        START,
        END,
        NR_TIMESTEPS
    )

    try:
        _ = run_windninja(cfg_path)
        print(f"ALL: WindNinja OK ({START:%Y-%m-%d %H:%M} → {END:%Y-%m-%d %H:%M}, {NR_TIMESTEPS} steps)")
    except Exception as e:
        print(f"ALL: WindNinja FAILED: {e}")
        try:
            print("--- CFG ---")
            print(cfg_path.read_text(encoding="utf-8"))
        except Exception:
            pass

    print("✅ Done.")

In [20]:
# THIS is only for one time period for an hour -> 6 timesteps!!!!

main()

ALL: WindNinja OK (2025-10-30 18:00 → 2025-10-30 20:50, 6 steps)
✅ Done.


#### 3.1 Parametrizing the main for multiple hour loops

In [18]:
def _ceil_to_hour(dt: datetime) -> datetime:
    if dt.minute == 0 and dt.second == 0 and dt.microsecond == 0:
        return dt
    return (dt.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1))

In [19]:
def run_hourly(period_start: datetime,
               period_end: datetime,
               sim_minutes: int = 50,
               timezone: str = TIMEZONE):
    """
    Run the pipeline at every full hour within [period_start, period_end),
    with each simulation window being sim_minutes long (default 50 min).

    Example:
      period_start = 2025-08-31 08:00 Europe/Zurich
      period_end   = 2025-09-01 08:00 Europe/Zurich
      runs at: 08:00, 09:00, 10:00, ...; each END = START + 50 min (capped at period_end).
    """
    if period_start.tzinfo is None:
        period_start = period_start.replace(tzinfo=tz.gettz(timezone))
    if period_end.tzinfo is None:
        period_end = period_end.replace(tzinfo=tz.gettz(timezone))

    # Start at the next full hour at/after period_start
    cur = _ceil_to_hour(period_start)

    count = 0
    while cur < period_end:
        start_i = cur
        end_i = min(cur + timedelta(minutes=sim_minutes), period_end)
        if end_i <= start_i:
            break  # nothing to do (e.g., final partial window too short)

        print(f"\n⏱ Running window {count+1}: {start_i:%Y-%m-%d %H:%M} → {end_i:%H:%M}")
        main_for_window(start_i, end_i)

        cur += timedelta(hours=1)
        count += 1

    print(f"\n✅ Finished {count} hourly windows from {period_start:%Y-%m-%d %H:%M} to {period_end:%Y-%m-%d %H:%M}.")

In [20]:
def main_for_window(START, END):
    BASE_OUT.mkdir(parents=True, exist_ok=True)

    # Ensure exactly one CSV per station/day; do not overwrite existing
    day_folder, daily_csvs = ensure_daily_files(engine, BASE_OUT, START, END, VIEWS)

    # Station list for THIS day only
    station_list_csv = create_station_list_csv(day_folder, START, END)

    # Output folder for the simulation span
    out_folder = make_span_folder(BASE_OUT, START, END)
    out_folder.mkdir(parents=True, exist_ok=True)

    # Build cfg — pointInitialization using the list CSV
    cfg_path = out_folder / f"wn_ALL_{START:%Y%m%d_%H%M}-{END:%Y%m%d_%H%M}.cfg"
    build_cfg_point_init_span_cli(
        cfg_path,
        out_folder,
        station_list_csv,
        START,
        END,
        NR_TIMESTEPS
    )

    try:
        _ = run_windninja(cfg_path)
        print(f"ALL: WindNinja OK ({START:%Y-%m-%d %H:%M} → {END:%Y-%m-%d %H:%M}, {NR_TIMESTEPS} steps)")
    except Exception as e:
        print(f"ALL: WindNinja FAILED: {e}")
        try:
            print("--- CFG ---")
            print(cfg_path.read_text(encoding='utf-8'))
        except Exception:
            pass

    print("✅ Done window.")

In [21]:
# run the big loop over a bigger period of time

# User-specified overall period (can be one day or many)
period_start = datetime(2025, 8, 1, 0, 0, tzinfo=tz.gettz(TIMEZONE))
period_end   = datetime(2025, 8,  30, 23, 50, tzinfo=tz.gettz(TIMEZONE))

# Run per hour, each with a 50-minute simulation window
run_hourly(period_start, period_end, sim_minutes=50)


⏱ Running window 1: 2025-08-01 00:00 → 00:50
ALL: WindNinja FAILED: WindNinja failed (4294967295). Output:
ERROR 1: Connection timed out after 5015 milliseconds
Exception caught: USER PROVIDED START TIME IS OUTSIDE DATASET TIME SPAN!! Provided start time < dataset start time for all wx stations!

--- CFG ---
num_threads = 8
elevation_file = merlingen_7.5mi.tif
initialization_method = pointInitialization
wx_station_filename = windninja_output/WXSTATIONS-CET-2025-08-01_0000-2025-08-01_0000-lake-thun/station_file_list_20250801.csv
match_points = true
number_time_steps = 6
start_year = 2025
start_month = 8
start_day = 1
start_hour = 0
start_minute = 0
stop_year = 2025
stop_month = 8
stop_day = 1
stop_hour = 0
stop_minute = 50
time_zone = Europe/Zurich
vegetation = trees
diurnal_winds = false
mesh_resolution = 150
units_mesh_resolution = m
output_wind_height = 10.0
units_output_wind_height = m
output_speed_units = mps
output_path = windninja_output/Output_2025-08-01_0000-0050_Google earth_

## 4. Test and compare windoutputs with GUI outputs

In [47]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

In [48]:
def compare_windninja_outputs(GUI_DIR: Path, CLI_DIR: Path, tolerance_pct: float = 1.0):
    """
    Compare outputs (KMZ, .ang.asc, .vel.asc) between GUI and CLI runs.
    
    Returns a DataFrame with:
      - filename_gui
      - filename_cli
      - same_output (True/False)
      - diff_pct (mean absolute % difference if numeric)
    
    Notes:
      - KMZ: compares ZIP content hashes (not unzipped rasters).
      - .asc: compares numeric grids after parsing to numpy arrays.
    """
    def read_asc_values(path: Path):
        """Read ASCII grid (.asc) file into numpy array of floats."""
        with open(path, "r", encoding="utf-8") as f:
            lines = f.readlines()
        # Skip header lines starting with known keywords
        data = []
        for line in lines:
            if any(line.lower().startswith(k) for k in ["ncols", "nrows", "xllcorner", "yllcorner", "cellsize", "nodata_value"]):
                continue
            vals = line.strip().split()
            if vals:
                data.extend(map(float, vals))
        return np.array(data)

    def compare_asc(gui_path: Path, cli_path: Path):
        """Compare two ASCII grids, return % difference."""
        a = read_asc_values(gui_path)
        b = read_asc_values(cli_path)
        if a.shape != b.shape:
            return np.nan
        # mean absolute percent difference ignoring zeros
        mask = (a != 0)
        if not mask.any():
            return 0.0
        diff_pct = np.mean(np.abs(a[mask] - b[mask]) / np.abs(a[mask])) * 100
        return diff_pct

    def compare_kmz(gui_path: Path, cli_path: Path):
        """Compare KMZ (zip) content by file list + CRCs."""
        try:
            with zipfile.ZipFile(gui_path, "r") as g, zipfile.ZipFile(cli_path, "r") as c:
                g_info = sorted([(x.filename, x.CRC) for x in g.infolist()])
                c_info = sorted([(x.filename, x.CRC) for x in c.infolist()])
                return g_info == c_info
        except Exception:
            return False

    results = []

    for gui_file in GUI_DIR.glob("**/*"):
        if not gui_file.is_file():
            continue

        rel = gui_file.relative_to(GUI_DIR)
        cli_file = CLI_DIR / rel

        if not cli_file.exists():
            results.append({
                "filename_gui": str(gui_file),
                "filename_cli": str(cli_file),
                "same_output": False,
                "diff_pct": np.nan
            })
            continue

        # Determine comparison type
        if gui_file.suffix.lower() == ".kmz":
            same = compare_kmz(gui_file, cli_file)
            diff = 0.0 if same else np.nan
        elif gui_file.suffix.lower().endswith(".asc"):
            diff = compare_asc(gui_file, cli_file)
            same = diff <= tolerance_pct
        else:
            # unknown file type, do binary compare
            same = gui_file.read_bytes() == cli_file.read_bytes()
            diff = 0.0 if same else np.nan

        results.append({
            "filename_gui": str(gui_file),
            "filename_cli": str(cli_file),
            "same_output": bool(same),
            "diff_pct": round(diff, 3) if diff is not None else np.nan
        })

    df = pd.DataFrame(results)
    return df

In [53]:
# === USER INPUT: point these to your *hour* folders (or parent dirs) ===
GUI_DIR = Path(r"C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min")
CLI_DIR = Path(r"C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min")

In [54]:
pd.set_option("display.max_colwidth", None)  # show full content, no truncation
pd.set_option("display.expand_frame_repr", False)  # prevent line wrapping
pd.set_option("display.max_columns", None)  # show all columns



df_cmp = compare_windninja_outputs(GUI_DIR, CLI_DIR, tolerance_pct=1.0)
display(df_cmp)

,filename_gui,filename_cli,same_output,diff_pct
0,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m.kmz,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m.kmz,False,NaN
1,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.asc,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.asc,True,0.0
2,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.asc.aux.xml,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.asc.aux.xml,False,NaN
3,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.prj,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_ang.prj,True,0.0
4,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_cld.asc,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_cld.asc,False,100.0
5,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_cld.prj,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_cld.prj,True,0.0
6,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.asc,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.asc,True,0.0
7,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.asc.aux.xml,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.asc.aux.xml,False,NaN
8,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.prj,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0600_150m_vel.prj,True,0.0
9,C:\Users\A\Documents\WINDNINJA\00_Output_2025-08-31_all day_Google earth_Fire_150m_10min\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0610_150m.kmz,C:\Users\A\Documents\XX_GitHub_Repo\data-waves\windninja\windninja_output\Output_2025-08-31_0600-0650_Google earth_Fire_150m_10min\merlingen_7.5mi_point_08-31-2025_0610_150m.kmz,False,NaN
